<h3>Analysing Food Security in Kenya</h3>

In [1]:
# import libraries
import pandas as pd
import pyreadstat as pr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import  stats
import warnings
import statsmodels.api as sm
import logging

In [2]:
# Ignore warnings
warnings.filterwarnings("ignore")

In [3]:
# Check the data directory
import os
# List all files and directories in the data folder
for root, dirs, files in os.walk("data"):
    for file in files:
        file_path = os.path.join(root, file)
        print(f"File: {file_path}")
      



File: data/SPSS Dataset Baseline Current.sav


In [4]:
# Set file path
SAV_FILE_PATH = "data/SPSS Dataset Baseline Current.sav"

# Create a folder for plots if it doesn't exist inorder to:
if not os.path.exists("plots"):
    os.makedirs("plots")

# Load the SPSS FILE
try:
    df, meta = pr.read_sav(SAV_FILE_PATH)
    print("✅ SAV file loaded successfully!")
    print(f"📊 Dataset shape: {df.shape}")
except Exception as e:
    print(f"❌ Error loading SAV file: {e}")
    exit()


# Basic information about the DataFrame
print("\n--- DataFrame Info ---")
df.info()


# First few rows to get a glimpse of the data
print("\n--- First 5 Rows ---")
print(df.head())


✅ SAV file loaded successfully!
📊 Dataset shape: (944, 547)

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Columns: 547 entries, Country to HOUSEHOLD_SIZE
dtypes: float64(527), object(20)
memory usage: 3.9+ MB

--- First 5 Rows ---
   Country  Studysite  HH_code  B_HH_type  B_HH_count  B_m1_gender  \
0      1.0       12.0  12009.0        1.0         7.0          1.0   
1      1.0       12.0  12008.0        3.0         5.0          2.0   
2      2.0       22.0  22177.0        1.0        14.0          1.0   
3      2.0       22.0  22265.0        1.0         7.0          1.0   
4      2.0       21.0  21226.0        1.0         8.0          1.0   

   B_m1_resp_age  B_m1_relation_head  B_m1_school_years  B_m1_home_occup  ...  \
0           62.0                 1.0                0.0              1.0  ...   
1           60.0                 1.0                0.0              1.0  ...   
2           39.0                 1.0                0.0

In [5]:

# Get descriptive statistics for all columns
print("\n--- Descriptive Statistics ---")
print(df.describe(include="all"))


--- Descriptive Statistics ---
           Country   Studysite       HH_code   B_HH_type  B_HH_count  \
count   944.000000  944.000000    944.000000  943.000000  944.000000   
unique         NaN         NaN           NaN         NaN         NaN   
top            NaN         NaN           NaN         NaN         NaN   
freq           NaN         NaN           NaN         NaN         NaN   
mean      1.549788   17.029661  17139.824153    1.193001    5.829449   
std       0.497779    5.041837   5055.312377    0.590866    2.048599   
min       1.000000   11.000000  11001.000000    1.000000    1.000000   
25%       1.000000   12.000000  12014.750000    1.000000    4.000000   
50%       2.000000   21.000000  21047.500000    1.000000    6.000000   
75%       2.000000   22.000000  22053.250000    1.000000    7.000000   
max       2.000000   22.000000  22289.000000    3.000000   15.000000   

        B_m1_gender  B_m1_resp_age  B_m1_relation_head  B_m1_school_years  \
count    944.000000     94

<h4>Accesing variable labels</h4>


In [6]:
# Print each column name alongside its descriptive label from the SPSS metadata
print("\n--- Variable Labels with Column Names ---")
for col_name, label in zip(meta.column_names, meta.column_labels):
    print(f"{col_name}: {label}")


--- Variable Labels with Column Names ---
Country: country of study site
Studysite: study site
HH_code: HH code
B_HH_type: HH type
B_HH_count: HH size
B_m1_gender: HH member1 gender
B_m1_resp_age: HH member1 age
B_m1_relation_head: HH member1 relation to HH head
B_m1_school_years: HH member1 school years
B_m1_home_occup: HH member1 at home
B_HH_m1_act_crop_farm: HH member1 crop farm
B_HH_m1_act_liv_keep: HH member1 keep livestock
B_HH_m1_act_poultry_keep: HH member1 keep poultry
B_HH_m1_act_salaried: HH member1 salary job
B_HH_m1_act_unemployed: HH member1 unemployed
B_m1_marital_status: HH member1 marital status
B_HH_m2_code: HH member2 code
B_m2_gender: HH member2 gender
B_m2_resp_age: HH member2 age
B_m2_relation_head: HH member2 relation to HH head
B_m2_school_years: HH member2 school years
B_m2_home_occup: HH member2 at home
B_m2_marital_status: HH member2 marital status
B_presence_child_6_59: index child aged 6-59m in HH
B_presence_mother_6_59: mother of child aged 6-59m availab

In [7]:
# Accesing individual labels
print("\n--- Variables with Value Labels ---")
print(list(meta.value_labels.keys()))



--- Variables with Value Labels ---
['labels0', 'labels1', 'labels2', 'labels3', 'labels4', 'labels5', 'labels6', 'labels7', 'labels8', 'labels9', 'labels10', 'labels11', 'labels12', 'labels13', 'labels14', 'labels15', 'labels16', 'labels17']


In [8]:
#  Print the contents of each label group
for key, value_map in meta.value_labels.items():
    print(f"{key}: {value_map}")

labels0: {1.0: 'Kenya', 2.0: 'Uganda'}
labels1: {11.0: 'West Pokot', 12.0: 'Turkana', 21.0: 'Napak', 22.0: 'Moroto'}
labels2: {1.0: 'male headed', 3.0: 'female headed', 6.0: 'child headed', 888.0: 'n.a.', 999.0: 'missing'}
labels3: {1.0: 'male', 2.0: 'female', 888.0: 'n.a.', 999.0: 'missing'}
labels4: {1.0: 'head', 2.0: 'spouse', 3.0: 'son', 4.0: 'daughter', 5.0: 'sister', 6.0: 'brother', 7.0: 'grandchild', 8.0: 'father', 9.0: 'mother', 66.0: 'other', 888.0: 'n.a.', 999.0: 'missing'}
labels5: {1.0: 'permanent home (10-12month/yr)', 2.0: 'freq.away (3-9m/yr)', 3.0: 'sometimes away(<3m/yr)=cat.1!'}
labels6: {0.0: 'no', 1.0: 'yes', 88.0: "don't know", 888.0: 'n.a.', 999.0: 'missing'}
labels7: {1.0: 'single', 2.0: 'monogamous', 3.0: 'polygamous', 4.0: 'divorced/separated', 5.0: 'widowed', 6.0: 'cohabiting (mono/polyg.)'}
labels8: {1.0: 'livestock', 2.0: 'crop', 3.0: 'livestock+crop', 4.0: 'other'}
labels9: {1.0: 'increase', 2.0: 'no change', 3.0: 'decrease'}
labels10: {1.0: 'male resp./hus

In [9]:
# Dictionary of all value labels
label_dictionary = {
    'labels0': {
        1.0: 'Kenya', 2.0: 'Uganda'
    },
    'labels1': {
        11.0: 'West Pokot', 12.0: 'Turkana', 21.0: 'Napak', 22.0: 'Moroto'
    },
    'labels2': {
        1.0: 'male headed', 3.0: 'female headed', 6.0: 'child headed',
        888.0: 'n.a.', 999.0: 'missing'
    },
    'labels3': {
        1.0: 'male', 2.0: 'female',
        888.0: 'n.a.', 999.0: 'missing'
    },
    'labels4': {
        1.0: 'head', 2.0: 'spouse', 3.0: 'son', 4.0: 'daughter', 5.0: 'sister',
        6.0: 'brother', 7.0: 'grandchild', 8.0: 'father', 9.0: 'mother',
        66.0: 'other', 888.0: 'n.a.', 999.0: 'missing'
    },
    'labels5': {
        1.0: 'permanent home (10-12month/yr)',
        2.0: 'freq.away (3-9m/yr)',
        3.0: 'sometimes away(<3m/yr)=cat.1!'
    },
    'labels6': {
        0.0: 'no', 1.0: 'yes',
        88.0: "don't know", 888.0: 'n.a.', 999.0: 'missing'
    },
    'labels7': {
        1.0: 'single', 2.0: 'monogamous', 3.0: 'polygamous',
        4.0: 'divorced/separated', 5.0: 'widowed',
        6.0: 'cohabiting (mono/polyg.)'
    },
    'labels8': {
        1.0: 'livestock', 2.0: 'crop',
        3.0: 'livestock+crop', 4.0: 'other'
    },
    'labels9': {
        1.0: 'increase', 2.0: 'no change', 3.0: 'decrease'
    },
    'labels10': {
        1.0: 'male resp./husband', 2.0: 'wife',
        3.0: 'both husband+wife', 66.0: 'other'
    },
    'labels11': {
        1.0: 'extremely', 2.0: 'much', 3.0: 'moderately',
        4.0: 'slightly', 5.0: 'not at all',
        888.0: 'n.a.', 999.0: '.'
    },
    'labels12': {
        1.0: 'Rarely (1-2 times)', 2.0: 'Sometimes (3-10 times)',
        3.0: 'Often (>10 times)'
    },
    'labels13': {
        1.0: 'Away', 2.0: 'Ill', 3.0: 'Dead'
    },
    'labels14': {
        1.0: 'severe', 2.0: 'moderate', 3.0: 'at risk',
        4.0: 'healthy', 888.0: 'n.a.'
    },
    'labels15': {
        1.0: 'severe-moderate, acute', 2.0: 'at risk', 3.0: 'healthy'
    },
    'labels16': {
        1.0: 'severe-moderate, acute', 2.0: 'at risk', 3.0: 'healthy'
    },
    'labels17': {
        1.0: 'underweight', 2.0: 'healthy',
        3.0: 'overweight', 4.0: 'obese'
    }
}


In [10]:
label = label_dictionary.get('labels3', {}).get(1.0, 'Unknown')
print(label)  # Output: male


male


In [11]:
# Loop through variables that have value labels and apply them directly
for column_name, label_mapping in meta.value_labels.items():
    if column_name in df.columns:
        df[f"{column_name}_label"] = df[column_name].map(label_mapping)
        print(f"Labeled column created: {column_name}_label")


In [12]:
# Save DataFrame with labeled columns to CSV
df.to_csv("labeled_data.csv", index=False)
print("Labeled data saved to labeled_data.csv")


Labeled data saved to labeled_data.csv


In [13]:

# --- Setup logging ---
logging.basicConfig(
    filename="data_analysis.log",
    level=logging.INFO,
    format='%(levelname)s: %(message)s'
)

# --- Load CSV with graceful error handling ---
"""
try:
    labelled_data = pd.read_csv("labeled_data.csv")
    logging.info("CSV file loaded successfully.")
    
except FileNotFoundError:
    logging.error("CSV file not found. Please check the file path.")
except pd.errors.ParserError:
    logging.error("Parsing error: File may be corrupt or not properly formatted.")
    
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")
"""

'\ntry:\n    labelled_data = pd.read_csv("labeled_data.csv")\n    logging.info("CSV file loaded successfully.")\n\nexcept FileNotFoundError:\n    logging.error("CSV file not found. Please check the file path.")\nexcept pd.errors.ParserError:\n    logging.error("Parsing error: File may be corrupt or not properly formatted.")\n\nexcept Exception as e:\n    logging.error(f"An unexpected error occurred: {e}")\n'

In [14]:
labelled_data = pd.read_csv("labeled_data.csv")
labelled_data.head()


,Country,Studysite,HH_code,B_HH_type,B_HH_count,B_m1_gender,B_m1_resp_age,B_m1_relation_head,B_m1_school_years,B_m1_home_occup,...,Q7F,Q8,Q8F,Q9,Q9F,HFIAS,MAHFP_total,EXPENDITURE_EXCEEDING_INCOME,HOUSEHOLD_HEAD_AGE,HOUSEHOLD_SIZE
0,1.0,12.0,12009.0,1.0,7.0,1.0,62.0,1.0,0.0,1.0,...,2.0,1.0,2.0,1.0,2.0,4.0,2.0,1.0,4.0,2.0
1,1.0,12.0,12008.0,3.0,5.0,2.0,60.0,1.0,0.0,1.0,...,2.0,1.0,3.0,1.0,2.0,4.0,2.0,0.0,3.0,1.0
2,2.0,22.0,22177.0,1.0,14.0,1.0,39.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,NaN,4.0,3.0,1.0,3.0,2.0
3,2.0,22.0,22265.0,1.0,7.0,1.0,58.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,4.0,2.0,0.0,3.0,2.0
4,2.0,21.0,21226.0,1.0,8.0,1.0,36.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,4.0,1.0,0.0,3.0,2.0


<h4>Data cleaning</h4>

In [ ]:
# Check for missing values across all columns​
print("\n--- Missing Values (%) ---")
#labelled_data.isna().sum()
# print missing values as percentage
missing_percentage = labelled_data.isnull().mean() * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))



--- Missing Values (%) ---
J_ch_food_type            100.000000
E_drink_water_lake_km     100.000000
E_drink_water_lake_min    100.000000
I_f_food_lack_disr_spc    100.000000
C_tree11_use_fruits        99.894068
                             ...    
J_food_no_eat               0.105932
J_comments_food             0.105932
K_f_necklace                0.105932
Q9                          0.105932
HFIAS                       0.105932
Length: 484, dtype: float64


In [20]:
# Identify columns with numerical data types​
numerical_cols = labelled_data.select_dtypes(include=np.number).columns.tolist()
numerical_cols

['Country',
 'Studysite',
 'HH_code',
 'B_HH_type',
 'B_HH_count',
 'B_m1_gender',
 'B_m1_resp_age',
 'B_m1_relation_head',
 'B_m1_school_years',
 'B_m1_home_occup',
 'B_HH_m1_act_crop_farm',
 'B_HH_m1_act_liv_keep',
 'B_HH_m1_act_poultry_keep',
 'B_HH_m1_act_salaried',
 'B_HH_m1_act_unemployed',
 'B_m1_marital_status',
 'B_HH_m2_code',
 'B_m2_gender',
 'B_m2_resp_age',
 'B_m2_relation_head',
 'B_m2_school_years',
 'B_m2_home_occup',
 'B_m2_marital_status',
 'B_presence_child_6_59',
 'B_presence_mother_6_59',
 'B_index_child_HH_m_code',
 'B_index_child_age',
 'C_HH_dep_full_agri',
 'C_HH_land_crop_ws_acc',
 'C_keep_animals',
 'C_used_keep_liv',
 'C_reason_drought',
 'C_reason_raid',
 'C_reason_poverty',
 'C_reason_Security',
 'C_reason_animal_dis',
 'C_reason_other',
 'C_cattle_keeping',
 'C_sheep_keeping',
 'C_goats_keeping',
 'C_camels_keeping',
 'C_chicken_keeping',
 'C_pigs_keeping',
 'C_liv_change_cattle',
 'C_liv_change_sheep',
 'C_liv_change_goats',
 'C_liv_change_camels',
 'C_l

In [21]:
# Exclude specific identifier columns or numerical codes
excluded_columns = ['HH_code', 'B_HH_type', 'Studysite']
numerical_cols = [col for col in numerical_cols if col not in excluded_columns]

# inpute with median
for col in numerical_cols:
    median_value = labelled_data[col].median()
    labelled_data[col].fillna(median_value, inplace=True)
    logging.info(f"Missing values in {col} filled with median: {median_value}")


In [22]:

# --- Example: Impute categorical missing values with the mode ---​
# Identify columns with object or categorical data types​
categorical_cols = labelled_data.select_dtypes(include=["object", "category"]).columns.tolist()
for col in categorical_cols:
    if labelled_data[col].isnull().any():
        mode_value = labelled_data[col].mode()[0]
        labelled_data[col].fillna(mode_value, inplace=True)
        logging.info(f"Missing values in {col} filled with mode: {mode_value}")

In [26]:
# Check for missing values in each column
missing_values = labelled_data.isnull().sum()

# Display columns with missing values
print(missing_values[missing_values > 0])

B_HH_type                   1
E_drink_water_lake_min    944
E_drink_water_lake_km     944
I_f_food_lack_disr_spc    944
J_ch_food_type            944
dtype: int64


In [ ]:
# Create a copy of the dataset
# drop columns with alot of missing values
labelled_data_copy = labelled_data.copy()

# Drop the specified columns from the copied dataset
columns_to_drop = ['E_drink_water_lake_min', 'E_drink_water_lake_km', 'I_f_food_lack_disr_spc', 'J_ch_food_type']
labelled_data_copy.drop(columns=columns_to_drop, inplace=True)

# Verify if the columns are dropped
print(labelled_data_copy.isnull().sum())

Country                         0
Studysite                       0
HH_code                         0
B_HH_type                       1
B_HH_count                      0
                               ..
HFIAS                           0
MAHFP_total                     0
EXPENDITURE_EXCEEDING_INCOME    0
HOUSEHOLD_HEAD_AGE              0
HOUSEHOLD_SIZE                  0
Length: 543, dtype: int64


The `B_HH_type` column still remaines and the data cleaning will have to continue with a closer examination of its details

In [32]:
# Impute missing values in B_HH_type with its median
if 'B_HH_type' in labelled_data_copy.columns:
    median_value = labelled_data_copy['B_HH_type'].median()
    labelled_data_copy['B_HH_type'].fillna(median_value, inplace=True)
    logging.info(f"Missing values in B_HH_type filled with median: {median_value}")

In [ ]:
# Check for missing values in each column
missing_values = labelled_data_copy.isnull().sum()

# Display columns with missing values
print(missing_values[missing_values > 0])

# Now there is no missing value in the dataset

Series([], dtype: int64)


<h5>Outlier detection is the process of identifying data points that significantly deviate from the rest of a dataset. These points can be errors or rare events and can heavily influence analyses, so detecting them is crucial.


</h5>
- I'll use the interquartile range, visualizations and machine learning to properly analysize outliers

Values outside this bracket - [Q1−1.5×IQR,Q3+1.5×IQR]

In [40]:
# Filter numerical_cols to include only columns present in the DataFrame
valid_numerical_cols = [col for col in numerical_cols if col in labelled_data_copy.columns]

# Detect outliers using the IQR method
outliers = {}
for col in valid_numerical_cols:
    Q1 = labelled_data_copy[col].quantile(0.25)
    Q3 = labelled_data_copy[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_data = labelled_data_copy[(labelled_data_copy[col] < lower_bound) | (labelled_data_copy[col] > upper_bound)]
    if not outlier_data.empty:
        outliers[col] = outlier_data

# Display the detected outliers
for col, data in outliers.items():
    print(f"Outliers detected in column '{col}':")
    print(data)

Outliers detected in column 'B_HH_count':
     Country  Studysite  HH_code  B_HH_type  B_HH_count  B_m1_gender  \
2        2.0       22.0  22177.0        1.0        14.0          1.0   
161      1.0       11.0  11043.0        1.0        14.0          1.0   
164      1.0       11.0  11046.0        1.0        15.0          1.0   
176      1.0       11.0  11058.0        1.0        14.0          1.0   
205      1.0       11.0  11087.0        1.0        12.0          1.0   
216      1.0       11.0  11098.0        1.0        12.0          1.0   
219      1.0       11.0  11101.0        1.0        12.0          1.0   
249      1.0       11.0  11131.0        1.0        12.0          1.0   
393      1.0       12.0  12072.0        1.0        12.0          1.0   
561      2.0       21.0  21064.0        1.0        13.0          1.0   

     B_m1_resp_age  B_m1_relation_head  B_m1_school_years  B_m1_home_occup  \
2             39.0                 1.0                0.0              1.0   
161      

In [41]:
# Display a summary
print("Outlier summary:")
for col, data in outliers.items():
    print(f"- {col}: {len(data)} outliers")

# Print top 5 outliers per column
for col, data in outliers.items():
    print(f"\nTop outliers in '{col}':")
    print(data[[col]].head())


Outlier summary:
- B_HH_count: 10 outliers
- B_m1_gender: 118 outliers
- B_m1_resp_age: 47 outliers
- B_m1_relation_head: 75 outliers
- B_m1_school_years: 93 outliers
- B_m1_home_occup: 82 outliers
- B_HH_m1_act_poultry_keep: 119 outliers
- B_HH_m1_act_salaried: 29 outliers
- B_HH_m1_act_unemployed: 51 outliers
- B_m1_marital_status: 51 outliers
- B_m2_gender: 73 outliers
- B_m2_resp_age: 92 outliers
- B_m2_relation_head: 165 outliers
- B_m2_school_years: 167 outliers
- B_m2_home_occup: 24 outliers
- B_m2_marital_status: 169 outliers
- B_presence_child_6_59: 185 outliers
- B_presence_mother_6_59: 50 outliers
- C_HH_land_crop_ws_acc: 63 outliers
- C_used_keep_liv: 34 outliers
- C_reason_drought: 23 outliers
- C_reason_raid: 40 outliers
- C_reason_poverty: 25 outliers
- C_reason_Security: 38 outliers
- C_reason_animal_dis: 92 outliers
- C_reason_other: 16 outliers
- C_goats_keeping: 146 outliers
- C_camels_keeping: 118 outliers
- C_pigs_keeping: 8 outliers
- C_liv_change_cattle: 173 outl